***Name:Mohammed junaid Iqbal Khalidi***

***USC ID : 1674712065***

***Github username: junaid537***




In [4]:
import numpy as np
import pandas as pd
import glob
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import bootstrap

### (a)Download the AReM data ###

***Downloaded from: https://archive.ics.uci.edu/ml/datasets/Activity+Recognition+system+based+on+Multisensor+data+fusion+(AReM)***


***I saw that dataset4.csv in bending2 folder is not the standard form of a csv file. I format that file by replacing space with comma in VS Code.***
***Also, there are additional comma/spaces for dataset9.csv and dataset14.csv in the cycling folder. I have manually removed them as they caused parsing errors.***



In [6]:
import pandas as pd
csv_file_path = '../data/AReM/cycling/dataset9.csv'
try:
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file_path,skiprows=5,header=None)
    print(df.shape)
except FileNotFoundError:
    print(f"Error: The file '{csv_file_path}' was not found.")
except pd.errors.ParserError:
    print("Error: There was a problem parsing the CSV file.")
except Exception as e:
    print(f"An error occurred: {e}")

(480, 7)


### (b) Keep datasets 1 and 2 in folders bending1 and bending 2, as well as datasets 1, 2, and 3 in other folders as test data and other datasets as train data. ###

In [8]:
#function to split testing and training data within a folder
def data_split(folder, total_files, test_files):
    # Make testing dataset
    test_set = []
    for i in range(1, test_files+1):
        path = '../data/AReM/' + folder + '/dataset' + str(i) + '.csv'
        df_test = pd.read_csv(path, skiprows = 4)  # Skip first 4 rows of information
        test_set.append(df_test)
    test_frame = pd.concat(test_set, ignore_index=True)
    
    # Make training dataset
    train_set = []
    for i in range(test_files+1, total_files+1):
        path = '../data/ARem/' + folder + '/dataset' + str(i) + '.csv'
        df_train = pd.read_csv(path, skiprows = 4)  # Skip first 4 rows of information
        train_set.append(df_train)
    train_frame = pd.concat(train_set, ignore_index=True)
    
    return test_frame, train_frame

In [10]:
# Generate testing and training data from ARem folder
# Initialize list to store information
testdata_list = list()
traindata_list = list()
# Splitting bending data into train and test
bending = ["bending1", "bending2"]
testdata_list.append(data_split("bending1", 7, 2)[0])
traindata_list.append(data_split("bending1", 7, 2)[1])
testdata_list.append(data_split("bending2", 6, 2)[0])
traindata_list.append(data_split("bending2", 6, 2)[1])
# Deal with other data
other = ["cycling", "lying", "sitting", "standing", "walking"]
for folder in other:
    testdata_list.append(data_split(folder, 15, 3)[0])
    traindata_list.append(data_split(folder, 15, 3)[1])
    
# Concatenate into testing and training dataframe
AREM_test = pd.concat(testdata_list, ignore_index=True)
AREM_train = pd.concat(traindata_list, ignore_index=True)

In [12]:
AREM_test

,# Columns: time,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23
0,0,39.25,0.43,22.75,0.43,33.75,1.30
1,250,39.25,0.43,23.00,0.00,33.00,0.00
2,500,39.25,0.43,23.25,0.43,33.00,0.00
3,750,39.50,0.50,23.00,0.71,33.00,0.00
4,1000,39.50,0.50,24.00,0.00,33.00,0.00
...,...,...,...,...,...,...,...
9115,118750,36.00,2.45,17.00,5.10,20.50,0.87
9116,119000,34.33,1.89,15.00,2.45,17.00,2.12
9117,119250,33.00,7.35,14.60,3.14,13.00,5.70
9118,119500,31.67,1.25,11.00,6.16,19.25,2.17


In [14]:
AREM_train

,# Columns: time,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23
0,0,42.00,0.71,21.25,0.43,30.00,0.00
1,250,41.50,0.50,20.25,1.48,31.25,1.09
2,500,41.50,0.50,14.25,1.92,33.00,0.00
3,750,40.75,0.83,15.75,0.43,33.00,0.00
4,1000,40.00,0.71,20.00,2.74,32.75,0.43
...,...,...,...,...,...,...,...
33114,118750,34.50,6.18,9.00,3.56,12.67,4.19
33115,119000,25.75,6.02,13.75,2.05,16.00,1.58
33116,119250,31.50,3.35,10.25,5.12,16.25,2.95
33117,119500,33.75,2.77,14.00,3.24,13.75,0.43


### c(i):Research what types of time-domain features are usually used in time series  ###
### classification and list them (examples are minimum, maximum, mean, etc) ###

Statistical features :

Mean ;
Standard deviation ;
Skewness, Kurtosis ;
Maximum and Minimum values ;
first quartile ;
third quartile ;
distribution ;
entropy ;
Autocorrelation ;
stationarity ;
Kurtosis ;
scaling properties ;



### (1-c-ii) Extract the time-domain features minimum, maximum, mean, median, standard deviation, first quartile, and third quartile ###
### for all of the 6 time series in each instance. You are free to normalize/standardize features or use them directly. ###

In [16]:
count_instances=0

In [18]:
# Set serie names
df=pd.DataFrame()
df = pd.concat([AREM_test, AREM_train], ignore_index=True)
df
series = ["avg_rss12", "var_rss12", "avg_rss13", "var_rss13", "avg_rss23", "var_rss23"]
index = [i for i in range(1, 7)]
# Set feature names to be extracted
operations=['min', 'max', 'mean', '50%', 'std', '25%', '75%']
colName = ['min', 'max', 'mean', 'median', 'std dev', '1st_quart', '3rd_quart']

resultant_df = pd.DataFrame({'Instance': range(1, 89)})
for i in range(1, 89):  #as total instances are 88
    for j in range(1,7): #as 6 columns
        for k in range(len(colName)):
            resultant_df[f'{colName[k]}{j}']=0
resultant_df = resultant_df.astype(float)
resultant_df['Instance'] = resultant_df['Instance'].astype(int)
#calculate("bending1", 7)
#calculate("bending2", 6)
#for folder_name in other:
#    calculate(folder_name, 15)
total_csv = list()
#Bending1:
for i in range(1, 8):
        path = '../data/AReM/' + 'bending1' + '/dataset' + str(i) + '.csv'
        total_csv.append(path)
for i in range(1, 7):
        path = '../data/AReM/' + 'bending2' + '/dataset' + str(i) + '.csv'
        total_csv.append(path)
for foldername in other:
    for i in range(1,16):
        path = '../data/AReM/' + foldername + '/dataset' + str(i) + '.csv'
        total_csv.append(path)
print(len(total_csv))
 
count_instances=0
for filename in total_csv:
        count_instances+=1
        df = pd.read_csv(filename, skiprows=4)
        for i in range(len(series)):
            min_value = df[series[i]].min()
            max_value=df[series[i]].max()
            mean_value=df[series[i]].mean()
            median_value=df[series[i]].median()
            stddev_value=df[series[i]].std()
            firstQ_value=df[series[i]].quantile(0.25)
            thirdQ_value=df[series[i]].quantile(0.75)
            condition=resultant_df['Instance']==count_instances
           
            resultant_df.loc[condition,f'min{i+1}']=min_value
            resultant_df.loc[condition,f'max{i+1}']=max_value
            resultant_df.loc[condition,f'mean{i+1}']=mean_value
            resultant_df.loc[condition,f'median{i+1}']=median_value
            resultant_df.loc[condition,f'std dev{i+1}']=stddev_value
            resultant_df.loc[condition,f'1st_quart{i+1}']=firstQ_value
            resultant_df.loc[condition,f'3rd_quart{i+1}']=thirdQ_value
display(resultant_df)

88


,Instance,min1,max1,mean1,median1,std dev1,1st_quart1,3rd_quart1,min2,max2,...,std dev5,1st_quart5,3rd_quart5,min6,max6,mean6,median6,std dev6,1st_quart6,3rd_quart6
0,1,37.25,45.00,40.624792,40.50,1.476967,39.25,42.0000,0.0,1.30,...,2.188449,33.0000,36.00,0.00,1.92,0.570583,0.43,0.582915,0.0000,1.3000
1,2,38.00,45.67,42.812812,42.50,1.435550,42.00,43.6700,0.0,1.22,...,1.995255,32.0000,34.50,0.00,3.11,0.571083,0.43,0.601010,0.0000,1.3000
2,3,35.00,47.40,43.954500,44.33,1.558835,43.00,45.0000,0.0,1.70,...,1.999604,35.3625,36.50,0.00,1.79,0.493292,0.43,0.513506,0.0000,0.9400
3,4,33.00,47.75,42.179812,43.50,3.670666,39.15,45.0000,0.0,3.00,...,3.849448,30.4575,36.33,0.00,2.18,0.613521,0.50,0.524317,0.0000,1.0000
4,5,33.00,45.75,41.678063,41.75,2.243490,41.33,42.7500,0.0,2.83,...,2.411026,28.4575,31.25,0.00,1.79,0.383292,0.43,0.389164,0.0000,0.5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,84,19.50,45.33,33.586875,34.25,4.650935,30.25,37.0000,0.0,14.67,...,3.283983,13.7300,18.25,0.00,8.32,3.259729,3.11,1.640243,2.0500,4.3225
84,85,19.75,45.50,34.322750,35.25,4.752477,31.00,38.0000,0.0,13.47,...,3.119856,13.5000,17.75,0.00,9.67,3.432563,3.20,1.732727,2.1575,4.5650
85,86,19.50,46.00,34.546229,35.25,4.842294,31.25,37.8125,0.0,12.47,...,2.823124,14.0000,17.75,0.00,10.00,3.338125,3.08,1.656742,2.1600,4.3350
86,87,23.50,46.25,34.873229,35.25,4.531720,31.75,38.2500,0.0,14.82,...,3.131076,13.7500,18.00,0.00,9.51,3.424646,3.27,1.690960,2.1700,4.5000


### c(iii):Estimate the standard deviation of each of the time-domain features you extracted from the data. Then, use Python’s bootstrapped or any other method to build a 90% bootsrap confidence interval for the standard deviation of each feature ###

In [20]:
resultant_df_est = resultant_df.loc[:, resultant_df.columns!="Instance"]


# Estimate the standard deviation for each feature
df_std = resultant_df_est.std()
display(df_std)

min1          9.569975
max1          4.394362
mean1         5.335718
median1       5.440054
std dev1      1.772153
1st_quart1    6.153590
3rd_quart1    5.138925
min2          0.000000
max2          5.062729
mean2         1.574164
median2       1.412244
std dev2      0.884105
1st_quart2    0.946386
3rd_quart2    2.125266
min3          2.956462
max3          4.875137
mean3         4.008380
median3       4.036396
std dev3      0.946710
1st_quart3    4.220658
3rd_quart3    4.171628
min4          0.000000
max4          2.183625
mean4         1.166114
median4       1.145586
std dev4      0.458242
1st_quart4    0.843620
3rd_quart4    1.552504
min5          6.124001
max5          5.741238
mean5         5.675593
median5       5.813782
std dev5      1.024898
1st_quart5    6.096465
3rd_quart5    5.531720
min6          0.045838
max6          2.518921
mean6         1.154812
median6       1.086474
std dev6      0.517617
1st_quart6    0.758584
3rd_quart6    1.523599
dtype: float64

***The standard deviation of each of the time-domain features we extracted are list above. We can observe that features like mean, median, 
and the stand deviation have relatively low standard deviation. Let's implement the bootstrap on features we extracted***

In [22]:
confidence_intervals = pd.DataFrame()
for i in range(len(resultant_df_est.columns)):
    sample_data = (resultant_df_est.iloc[:, i],)
    res = bootstrap(sample_data, np.std, confidence_level = 0.9, random_state = 1, method = 'percentile')
    temp = pd.DataFrame({'Feature': resultant_df_est.columns[i], 'Lower Bound': res.confidence_interval.low, 'Upper Bound': res.confidence_interval.high}, index=[i+1])
    confidence_intervals = pd.concat([confidence_intervals, temp], ignore_index = True)
confidence_intervals

,Feature,Lower Bound,Upper Bound
0,min1,8.227103,10.687351
1,max1,3.302748,5.264298
2,mean1,4.675269,5.842371
3,median1,4.761417,5.966529
4,std dev1,1.564920,1.941307
5,1st_quart1,5.541772,6.620566
6,3rd_quart1,4.298619,5.795733
7,min2,0.000000,0.000000
8,max2,4.604009,5.378704
9,mean2,1.389844,1.695784


***(iv):Select Features :***

From the above summary data, according to me, the 3 most important time domain features:

Mean: Mean of the points can inform us about the central tendency of the data, taking various outliers into consideration.

Median: Since the data has too many outliers which can make the data skewed, median could be a good feature to have which is not affected much by outliers.

Standard deviation: Standard deviation tells us that roughly how many datapoints are away from the mean. It gives an insight into the underlying data distribution.

### ISLR 3.7.4 ###

### (a) Linear Train ###


***The polynomial regression will have a lower training RSS than the linear regression because it is more complex and could fit toughter against data.
The cubic regression model will have better flexibility and will better fit to the training data, hence lead to a lower RSS value as compared to the RSS value for linear regression.***

***(b) The polynomial regression (with a high probability) wiil have a higher test RSS as the overfit from training would have more error than the 
linear regression.***

***(c)  Not Linear Train :
RSS for cubic regression would be smaller as compared to RSS for linear regression.
The principle remains the same, the cubic regression model has more flexibility and will be able to better fit the t
raining data and hence will have a lower RSS value as compared to the linear regression model.***

***(d):The information provided is insufficient.For test data, the RSS would depend on the relationship of the data, if the data is closer to linear, than linear will give lower RSS than cubic regression.
Otherwise, if the data is far from linear, cubic will give lower RSS than linear regression.***